<a href="https://colab.research.google.com/github/mantoan-thi/Wire-harness-analysis/blob/main/An%C3%A1lise_TE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Instação de biblioteca para trabalhar com Excel

In [ ]:
!pip install xlsxwriter

# Nova secção

In [187]:
import pandas as pd
import numpy as np
from timeit import default_timer as timer
from datetime import timedelta
import warnings
warnings.filterwarnings("ignore")

Carregando dados

In [208]:
df_circuitos = pd.read_excel('/content/Consulta BOM v17.xlsm',sheet_name='CIRCUITO')

Selecionando somentes colunas importantes

In [183]:
# Selecionando somentes colunas importantes
def selecionar_colunas(df_circuitos):
  if 'OT_DISCRIMINADA' in df_circuitos.columns:
    df_circuitos = df_circuitos[['PROD_TERM', 'OT', 'OT_DISCRIMINADA', 'CIRC_ESQ','CIR_DIR', 'TIPO', 
                                'BITOLA', 'COR', 'COMP', 'COD_OP_01_ESQ', 'TERMINAL_ESQ','ACC_ESQ_1',
                                'COD_OP_01_DIR','TERMINAL_DIR','ACC_DIR_1','SOLDA']].copy()

    # Obtenha os nomes dos índices para os quais a coluna 'OT_DISCRIMINADA' tem o valor SIM

    index_ot_discriminada = df_circuitos[df_circuitos['OT_DISCRIMINADA']=='SIM'].index
    if index_ot_discriminada.shape[0] > 0: # Excluindo esses índices de linha de dataFrame
      df_circuitos.drop(index_ot_discriminada, inplace=True)
      df_circuitos.drop(columns=['OT_DISCRIMINADA'],axis=1, inplace=True)
    df_circuitos.fillna(0,inplace=True)

  df_circuitos['COD_OP_01_ESQ'] = df_circuitos['COD_OP_01_ESQ'].apply(lambda x: 0 if x != 'U' else 'U')
  df_circuitos['COD_OP_01_DIR'] = df_circuitos['COD_OP_01_DIR'].apply(lambda x: 0 if x != 'U' else 'U')
  df_circuitos.head()
  # Quantidade de circuitos por part number
  lista_pn = []
  pn_anterior = 0
  for pn in df_circuitos['PROD_TERM']:
    if pn_anterior != pn:
      pn_anterior = pn
      qtde = df_circuitos[df_circuitos['PROD_TERM']==pn].shape[0]
      lista_pn.append([pn,qtde])
  lista_pn =pd.DataFrame(lista_pn,columns=['Part_number','Qtde'])
  lista_pn.sort_values(by=['Qtde'],ascending=False,inplace=True)
  lista_pn.reset_index(inplace=True,drop=True)


  qtde_pn = lista_pn.shape[0]
  menor_pn = lista_pn['Part_number'][-1:].values[0]
  maior_pn = lista_pn['Part_number'][0]
  Describ = lista_pn.describe()
  return lista_pn,df_circuitos


Comparando

In [85]:
def analisar(list_comp,tol):
  contar1 = 0
  contar2 = 0
  list_comp['Delta'] = 0
  while contar1 < list_comp.shape[0]:
    contar2 = contar1+1
    while contar2 < list_comp.shape[0]:
      if list_comp['PROD_TERM'].iloc[contar1]!=list_comp['PROD_TERM'].iloc[contar2]:
        # Tipo+bit+cor
        tipoA = list_comp['TIPO'].iloc[contar1]+list_comp['BITOLA'].iloc[contar1]+list_comp['COR'].iloc[contar1]
        tipoB = list_comp['TIPO'].iloc[contar2]+list_comp['BITOLA'].iloc[contar2]+list_comp['COR'].iloc[contar2]
        TAE = str(list_comp['COD_OP_01_ESQ'].iloc[contar1])+str(list_comp['TERMINAL_ESQ'].iloc[contar1])+str(list_comp['ACC_ESQ_1'].iloc[contar1])
        TAD = str(list_comp['COD_OP_01_DIR'].iloc[contar1])+str(list_comp['TERMINAL_DIR'].iloc[contar1])+str(list_comp['ACC_DIR_1'].iloc[contar1])
        TBE = str(list_comp['COD_OP_01_ESQ'].iloc[contar2])+str(list_comp['TERMINAL_ESQ'].iloc[contar2])+str(list_comp['ACC_ESQ_1'].iloc[contar2])
        TBD = str(list_comp['COD_OP_01_DIR'].iloc[contar2])+str(list_comp['TERMINAL_DIR'].iloc[contar2])+str(list_comp['ACC_DIR_1'].iloc[contar2])
        COMPA = int(list_comp['COMP'].iloc[contar1])
        COMPB = int(list_comp['COMP'].iloc[contar2])
        dif = abs(COMPA-COMPB)
        JTA = str(list_comp['SOLDA'].iloc[contar1])[0]
        JTB = str(list_comp['SOLDA'].iloc[contar2])[0]
        if tipoA == tipoB:                            # Comparando Tipo,Bit e cor
          if COMPA == COMPB or dif < tol:              # Checando dimensional
            if TAE==TBE and TAD==TBD:                  # Terminais
              list_comp['Delta'].iloc[contar1] ='Comum' 
              list_comp['Delta'].iloc[contar2] ='Comum'
            if TAE==TBD and TAD==TBE:                  # Terminais
              list_comp['Delta'].iloc[contar1] ='Comum' 
              list_comp['Delta'].iloc[contar2] ='Comum'

      contar2 +=1
    contar1+=1
    contar2 = 0
    list_comp['Delta'].loc[list_comp['Delta']==0]='DIF'
    lista_analisada = list_comp.sort_values(by=['OT'],ascending=True,inplace=True)
  return list_comp

In [102]:
def contar_saidas(x):
  valor = 0
  for e in nova_lista3['Saida_esq']:
    if e == x:
      valor +=1
  for d in nova_lista3['Saida_dir']:
    if d == x:
      valor +=1
  return valor

In [103]:
def contar_lig(e,d):
  contador = 0
  valor = 0
  while contador < nova_lista3.shape[0]:
    if nova_lista3['Saida_esq'].loc[contador:contador].any() == e and nova_lista3['Saida_dir'].loc[contador:contador].any()==d:
      valor +=1
    if nova_lista3['Saida_esq'].loc[contador:contador].any() == d and nova_lista3['Saida_dir'].loc[contador:contador].any() == e:
      valor +=1
    contador +=1
  return valor

In [104]:
def perct(l,c):
  return'{:.0%}'.format(l/c)

In [105]:
def processar(new_df_circuitos):
  # separa as saídas dos circuitos
  new_df_circuitos['Saida_esq'] = new_df_circuitos['CIRC_ESQ'].str[-3:]
  new_df_circuitos['Saida_dir'] = new_df_circuitos['CIR_DIR'].str[-3:]

  # Calcula a quantidade de saídas
  new_df_circuitos['qtde_sds_esq'] = new_df_circuitos['Saida_esq'].apply(contar_saidas)
  new_df_circuitos['qtde_sds_dir'] = new_df_circuitos['Saida_dir'].apply(contar_saidas)

  # Calcula a quantidade de ligações
  new_df_circuitos['qtde_lig_esq'] = new_df_circuitos.apply(lambda x: contar_lig(x['Saida_esq'],x['Saida_dir']), axis=1)
  new_df_circuitos['qtde_lig_dir'] = new_df_circuitos['qtde_lig_esq']

  # Calcula o percetual de ligações

  new_df_circuitos['Perct_lig_esq'] = new_df_circuitos.apply(lambda x: perct(x['qtde_lig_esq'],x['qtde_sds_esq']), axis=1)
  new_df_circuitos['Perct_lig_dir'] = new_df_circuitos.apply(lambda x: perct(x['qtde_lig_dir'],x['qtde_sds_dir']), axis=1)
  return new_df_circuitos

In [209]:
# Calcular a quantidade de comparações
lista_pn,df = selecionar_colunas(df_circuitos)
contador1 =0
contador2 = 0
lista_de_comparacao = []
Total = 0

cont = 0
qtde_saidaA = 0
qtde_saidaB = 0
lista_de_saidasA = []
lista_de_saidasB = []
qtde_ckt_A = []
qtde_ckt_B = []
resultado = []
while contador1 <= lista_pn.shape[0]:
  contador2=contador1+1
  while contador2 < lista_pn.shape[0]:

    pnA = lista_pn['Part_number'][contador1]
    pnB = lista_pn['Part_number'][contador2]

    list_comp = df[(df['PROD_TERM'] == pnA) | (df['PROD_TERM']==pnB)]
    nova_lista = analisar(list_comp,50)
    nova_lista['Total'] = 1
    nova_lista['Delta'].loc[(nova_lista['Delta']=='DIF') & (nova_lista['PROD_TERM']==pnA)]='Adicionar'
    nova_lista['Delta'].loc[(nova_lista['Delta']=='DIF') & (nova_lista['PROD_TERM']==pnB)]='Excluir'

    nova_lista1 = nova_lista.copy()
    nova_lista2 = nova_lista.copy()
    nova_lista3 = pd.pivot_table(nova_lista1,index=['OT', 'CIRC_ESQ', 'CIR_DIR', 'TIPO', 'BITOLA', 'COR',
       'COMP', 'COD_OP_01_ESQ', 'TERMINAL_ESQ', 'ACC_ESQ_1', 'COD_OP_01_DIR',
       'TERMINAL_DIR', 'ACC_DIR_1', 'SOLDA','Delta'],values=["Total"],columns=['PROD_TERM'],aggfunc=[np.sum],fill_value=0,)
    
    nova_lista3.columns = [col[2] for col in nova_lista3.columns]
    nova_lista3.reset_index(inplace=True)

    nova_lista3[pnA] = nova_lista3[pnA].apply(lambda x: pnA if x == 1 else 0)
    nova_lista3[pnB] = nova_lista3[pnB].apply(lambda x: pnB if x == 1 else 0)
    nova_lista4 = processar(nova_lista3)
    while cont < nova_lista4.shape[0]:
      if nova_lista4['Delta'][cont]!='Comum' and nova_lista4['Perct_lig_esq'][cont] == '100%':
        if nova_lista4[pnA][cont]!=0:
          qtde_saidaA+=1
          lista_de_saidasA.append(nova_lista4['Saida_esq'][cont])
          print(lista_de_saidasA)
          qtde_ckt_A.append(nova_lista4['qtde_sds_esq'][cont])
        else:
          qtde_saidaB+=1
          lista_de_saidasB.append(nova_lista4['Saida_esq'][cont])
          qtde_ckt_B.append(nova_lista4['qtde_sds_dir'][cont])
      if nova_lista4['Delta'][cont]!='Comum' and nova_lista4['Perct_lig_dir'][cont] == '100%':
        if nova_lista4[pnA][cont]!=0:
          qtde_saidaA+=1
          lista_de_saidasA.append(nova_lista4['Saida_dir'][cont])
          qtde_ckt_A.append(nova_lista4['qtde_sds_esq'][cont])
        else:
          qtde_saidaB+=1
          lista_de_saidasB.append(nova_lista4['Saida_dir'][cont])
          qtde_ckt_B.append(nova_lista4['qtde_sds_dir'][cont])
      cont+=1
    resultado.append([pnA,qtde_saidaA,', '.join(lista_de_saidasA),qtde_ckt_A,pnB,qtde_saidaB,', '.join(lista_de_saidasB),qtde_ckt_B])

    lista_de_comparacao.append(lista_pn['Part_number'][contador1]+' vs '+lista_pn['Part_number'][contador2])
    contador2+=1
  contador1+=1
  contador2 = 0
#lista_de_comparacao = pd.DataFrame(lista_de_comparacao,columns=['Part Numbers'])

Apresentando Resultados

In [210]:
pd.DataFrame(resultado,columns=['PNA','Núm_saídas_diferentes','Saídas_exclusivas','Núm_CktA','PNB','Núm_saídas_diferentesB','Saídas_exclusivasB','Núm_CktB'])

,PNA,Núm_saídas_diferentes,Saídas_exclusivas,Núm_CktA,PNB,Núm_saídas_diferentesB,Saídas_exclusivasB,Núm_CktB
0,82111F5F10 A00,0,,[],82115F5R70 A00,0,,[]
1,82111F5F10 A00,0,,[],82111F5F20 000,0,,[]
2,82111F5F10 A00,0,,[],82115F5R80 A00,0,,[]
3,82115F5R70 A00,0,,[],82111F5F20 000,0,,[]
4,82115F5R70 A00,0,,[],82115F5R80 A00,0,,[]
5,82111F5F20 000,0,,[],82115F5R80 A00,0,,[]


In [213]:
nova_lista4[(nova_lista4['Delta']!='Comum') & (nova_lista4['Perct_lig_dir']=='100%')]

,OT,CIRC_ESQ,CIR_DIR,TIPO,BITOLA,COR,COMP,COD_OP_01_ESQ,TERMINAL_ESQ,ACC_ESQ_1,COD_OP_01_DIR,TERMINAL_DIR,ACC_DIR_1,SOLDA,Delta,82111F5F20 000,82115F5R80 A00,Saida_esq,Saida_dir,qtde_sds_esq,qtde_sds_dir,qtde_lig_esq,qtde_lig_dir,Perct_lig_esq,Perct_lig_dir


In [214]:
nova_lista4

,OT,CIRC_ESQ,CIR_DIR,TIPO,BITOLA,COR,COMP,COD_OP_01_ESQ,TERMINAL_ESQ,ACC_ESQ_1,COD_OP_01_DIR,TERMINAL_DIR,ACC_DIR_1,SOLDA,Delta,82111F5F20 000,82115F5R80 A00,Saida_esq,Saida_dir,qtde_sds_esq,qtde_sds_dir,qtde_lig_esq,qtde_lig_dir,Perct_lig_esq,Perct_lig_dir
0,8,VBM 730,W8U 314,CIVUS,0.35,LG,1014,0,7116-4727-02,7172-5032-20,0,7116-3153-02,7406-3000,0,Adicionar,82111F5F20 000,0,730,314,21,21,5,5,24%,24%
1,9,VBN 731,W8V 314,CIVUS,0.35,L,1143,0,7116-4727-02,7172-5032-20,0,7116-3153-02,0,0,Excluir,0,82115F5R80 A00,731,314,17,21,4,4,24%,19%
2,10,VBN 730,W8V 314,CIVUS,0.35,L,1014,0,7116-4727-02,7172-5032-20,0,7116-3153-02,7406-3000,0,Adicionar,82111F5F20 000,0,730,314,21,21,5,5,24%,24%
3,11,E70 314,VBP 731,CIVUS,0.35,Y,1143,0,7116-3153-02,0,0,7116-4727-02,7172-5032-20,0,Excluir,0,82115F5R80 A00,314,731,21,17,4,4,19%,24%
4,12,E70 314,VBP 730,CIVUS,0.35,Y,1014,0,7116-3153-02,7406-3000,0,7116-4727-02,7172-5032-20,0,Adicionar,82111F5F20 000,0,314,730,21,21,5,5,24%,24%
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
172,295,K97 501,K7L 710,LFAVSS,2.0F,L,691,0,7116-1686-02,0,0,7116-4037,7158-3015-10,0,Comum,82111F5F20 000,82115F5R80 A00,501,710,31,2,1,1,3%,50%
173,297,E11 501,E29 501,CIVUS,0.5,B,150,U,7116-2892-02,0,0,7116-1685-02,0,0,Comum,82111F5F20 000,82115F5R80 A00,501,501,31,31,8,8,26%,26%
174,298,OCJ 315,WZE 060,LFAVSS,0.3,V,1120,0,7116-3154-08,0,0,7116-4028-08,7158-3006-90,0,Comum,82111F5F20 000,82115F5R80 A00,315,060,7,3,3,3,43%,100%
175,299,OCK 315,WZL 060,LFAVSS,0.3,BE,1120,0,7116-3154-08,0,0,7116-4028-08,7158-3006-90,0,Comum,82111F5F20 000,82115F5R80 A00,315,060,7,3,3,3,43%,100%
